# 1. Send data from API to Kafka

In [18]:
import json
import requests
from kafka import KafkaProducer
import time

# === CONFIGURATION ===
API_URL = "https://api.aviationstack.com/v1/flights"
API_KEY = "d0c318dffed1d4677589349b81334616"
TOPIC_NAME = "flights_data"
KAFKA_BROKERS = ["localhost:29092", "localhost:29093", "localhost:29094"]

# === FETCH DATA ===
def fetch_flight_data():
    params = {
        'access_key': API_KEY,
    }
    response = requests.get(API_URL, params=params)
    response.raise_for_status()
    data = response.json()
    return data.get('data', [])

# === PRODUCE TO KAFKA ===
def send_to_kafka(flight_data):
    producer = KafkaProducer(
        bootstrap_servers=KAFKA_BROKERS,
        value_serializer=lambda v: json.dumps(v).encode('utf-8')
    )
    
    for record in flight_data:
        print(f"Sending record to Kafka: {record}")
        # Send each flight record to the Kafka topic
        producer.send(TOPIC_NAME, value=record)
    
    producer.flush()
    producer.close()

# === MAIN ===
while True:
    print("Fetching flight data...")
    try:
        flights = fetch_flight_data()
        print(flights)
        if flights:
            print(f"Fetched {len(flights)} records. Sending to Kafka...")
            send_to_kafka(flights)
            print("All data sent to Kafka.")
        else:
            print("No flight data fetched.")
    except requests.RequestException as e:
        print(f"Error fetching flight data: {e}")
    except Exception as e:
        print(f"An error occurred: {e}")
    time.sleep(60)  # Wait for 1 minute before fetching again


Fetching flight data...
[{'flight_date': '2025-04-28', 'flight_status': 'scheduled', 'departure': {'airport': 'Singapore Changi', 'timezone': 'Asia/Singapore', 'iata': 'SIN', 'icao': 'WSSS', 'terminal': '1', 'gate': 'D35', 'delay': None, 'scheduled': '2025-04-28T04:50:00+00:00', 'estimated': '2025-04-28T04:50:00+00:00', 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'arrival': {'airport': 'Kuala Lumpur International Airport (klia)', 'timezone': 'Asia/Kuala_Lumpur', 'iata': 'KUL', 'icao': 'WMKK', 'terminal': '2', 'gate': None, 'baggage': None, 'scheduled': '2025-04-28T06:00:00+00:00', 'delay': None, 'estimated': None, 'actual': None, 'estimated_runway': None, 'actual_runway': None}, 'airline': {'name': 'Singapore Airlines', 'iata': 'SQ', 'icao': 'SIA'}, 'flight': {'number': '8578', 'iata': 'SQ8578', 'icao': 'SIA8578', 'codeshared': {'airline_name': 'scoot', 'airline_iata': 'tr', 'airline_icao': 'tgw', 'flight_number': '456', 'flight_iata': 'tr456', 'flight_icao': 'tgw

KeyboardInterrupt: 

FileNotFoundError: [Errno 2] No such file or directory: 'java'